# AgentX Quickstart

Welcome to the XEntropy quickstart guide, your first step in exploring the world of application development with agent-based large language models (or in short, agentic llm).

XEntropy is an innovative development platform designed to elevate the ease at which developers can construct intelligent applications. Throughout this tutorial, we will navigate the following key areas:

1. **Account creation:** We guide you step-by-step in establishing your XEntropy account:the gateway to a host of agentic llm tools.

2. **Loading tools from XEntropy:** Learn how to seamlessly and efficiently import tools from the platform into your environment to kickstart your foray into developing llm agents and intelligent applications.

3. **Managing your account:** Your XEntropy account is useful for many things. Learn how to:
    - **Purchase credit**: Convenience and accessibility are core to XEntropy. Understand how to replenish your account balance in a straightforward, user-friendly manner.

   - **Get paid**: Having created applications or tools that other developers find useful? Here's how to benefit from your creations.

   - **Modify and delete tools**: As your skill level advances, so might your tools and applications. Learn how to adjust and remove tools from your portfolio.

   - **Utilisation of logs**: Track your progress and rectify mistakes by understanding how to make use of logs effectively.

   - **Finetune your own agentic llm**: Capture the essence of your application development ideation by discovering how to fine-tune your personal agentic llm to reach peak performance.

## Installation

In [1]:
# Install AgentX
from distutils.dir_util import copy_tree, remove_tree
import os
import sys

site_packages_path = [x for x in sys.path if x.endswith('site-packages')][0]
# remove any existing installation
if os.path.exists(f'{site_packages_path}/agentx'):
    remove_tree(f'{site_packages_path}/agentx')
copy_tree(f'../agentx', f'{site_packages_path}/agentx')

['/home/jay/.local/lib/python3.12/site-packages/agentx/__init__.py',
 '/home/jay/.local/lib/python3.12/site-packages/agentx/__pycache__/__init__.cpython-310.pyc',
 '/home/jay/.local/lib/python3.12/site-packages/agentx/__pycache__/agent.cpython-310.pyc',
 '/home/jay/.local/lib/python3.12/site-packages/agentx/__pycache__/bedrock_client.cpython-310.pyc',
 '/home/jay/.local/lib/python3.12/site-packages/agentx/__pycache__/client.cpython-310.pyc',
 '/home/jay/.local/lib/python3.12/site-packages/agentx/__pycache__/groupchat.cpython-310.pyc',
 '/home/jay/.local/lib/python3.12/site-packages/agentx/__pycache__/oai_client.cpython-310.pyc',
 '/home/jay/.local/lib/python3.12/site-packages/agentx/__pycache__/optimisers.cpython-310.pyc',
 '/home/jay/.local/lib/python3.12/site-packages/agentx/__pycache__/saved_agents.cpython-310.pyc',
 '/home/jay/.local/lib/python3.12/site-packages/agentx/__pycache__/schema.cpython-310.pyc',
 '/home/jay/.local/lib/python3.12/site-packages/agentx/__pycache__/tool.cpyth

In [2]:
# Install dependencies to complete this tutorial
%pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## Create your account

In [3]:
from agentx.client import Client

# load environment variables
from dotenv import load_dotenv

load_dotenv()

from agentx.tool import Tool
import os

tool_search = Tool.load(
    'xentropy:tool_search', 
    api_key=os.environ.get('XENTROPY_API_KEY')
)

# display the function call schema
print(tool_search.input_json_schema)

from rich import print as rich_print
import json
# run the tool
rich_print(
    json.loads(
        tool_search.run(query='convert an address to geolocation')
    )
)

# Get a tool that converts address to latitude longitude cooself.credentials.tokenrdinate
geocoding = Tool.load('xentropy:geocoding', api_key=os.environ.get('XENTROPY_API_KEY'))
# Get a tool that computes the earth surface distance between two coordinates
geodesic = Tool.load('xentropy:geodesic', api_key=os.environ.get('XENTROPY_API_KEY'))

{'properties': {'query': {'title': 'Query', 'type': 'string'}, 'top_k': {'default': 16, 'maximum': 16, 'minimum': 1, 'title': 'Top K', 'type': 'integer'}}, 'required': ['query'], 'title': 'SearchQuery', 'type': 'object'}


[
    {
        'id': 'xentropy:geocoding',
        'description': 'Look up address based on longitude and latitude coordinate.',
        'cos_similarity': 0.864535451
    },
    {
        'id': 'xentropy:geodesic',
        'description': 'Calculate the earth surface distance between two latitude and longitude coordinate',
        'cos_similarity': 0.791891694
    },
    {
        'id': 'xentropy:restaurant_search',
        'description': 'Search restaurants based on food preference, location, and other user requirements. 
Examples: {"include_type": "korean_restaurant", "query": "Korean barbeque in Mountain View with spacious seating 
and good service"}',
        'cos_similarity': 0.710569739
    },
    {
        'id': 'justintime:mark_dse_question',
        'description': "Mark the students' answers for the given question",
        'cos_similarity': 0.67601645
    },
    {
        'id': 'xentropy:tool_search',
        'description': 'Search for tools that accomplish a specific task.',
        'cos_similarity': 0.675112486
    },
    {
        'id': 'xentropy:tool_search_paid',
        'description': 'Search for tools that accomplish a specific task.',
        'cos_similarity': 0.675112486
    },
    {
        'id': 'justintime:generate_dse_question',
        'description': "Generate a question based on the given topics and bloom's taxonomy",
        'cos_similarity': 0.65435487
    }
]

## Creating Agents with the loaded tools

In [4]:
from agentx.agent import Agent
from agentx.schema import GenerationConfig, Message, Content
from typing import List

tools = [geocoding, geodesic]

generation_config = GenerationConfig(
    api_type='vertexai',
    path_to_google_service_account_json='vertex_ai_creds.json',
    google_application_credential_scope=['https://www.googleapis.com/auth/cloud-platform'],
    region='asia-northeast3',
)

""" generation_config = GenerationConfig(
    api_type='azure',
    api_key=os.environ.get('AZURE_OPENAI_KEY'),
    api_version="2024-02-15-preview",
    base_url="https://genai-instance-test-jeremy.openai.azure.com/", 
)  """

# replace with your own azure deployment to choose different models
generation_config.model = 'gemini-pro'
# generation_config.azure_deployment = 'gpt-35-jeremy'

# define a termination function
# once the agent execute the geodesic tool, the conversation ends
def terminate(history:List[Message]):
    tool_response_name = [
        message.content.tool_response.name for message in history if message.content.tool_response
    ]
    if 'xentropy--geodesic' in tool_response_name:
        return True
    return False
    
# define the agent
agent = Agent(
    name='agent',
    generation_config=generation_config,
    system_prompt=None,# Gemini doesn't support context or system prompts
    tools = tools,
    termination_function=terminate,
)

In [5]:
# the agent is ready to generate messages
messages = [
    Message(
        role='user',
        content=Content(
            text='What is the distance between Kalinga Vihar, Bubaneswar and Vartak Nagar Thane?',
        ),
    )
]

In [6]:
# use the agent to use geocoding and geodesic to find earth surface distance between two locations

max_iterations = 1
 
for i in range(max_iterations):
    response = agent.generate_response(messages)
    # termination condition is met
    if response == None:
        break
    else:
        [rich_print(r.model_dump(exclude_unset=True)) for r in response]

{'address': 'Kalinga Vihar, Bubaneswar'}
{'address': 'Vartak Nagar Thane'}
{'coordinate_1': {'latitude': 19.2083, 'longitude': 72.961}, 'coordinate_0': {'latitude': 20.2365, 'longitude': 85.7557}}


{
    'role': 'model',
    'content': {
        'tool_calls': [
            {
                'id': 'geocoding',
                'type': 'function',
                'function_call': {'name': 'geocoding', 'arguments': '{"address": "Kalinga Vihar, Bubaneswar"}'}
            }
        ]
    },
    'name': 'agent'
}

{
    'role': 'tool',
    'content': {
        'tool_response': {
            'id': 'geocoding',
            'name': 'geocoding',
            'content': '{"latitude":20.2364668,"longitude":85.7556542}'
        }
    },
    'name': 'geocoding'
}

{
    'role': 'model',
    'content': {
        'tool_calls': [
            {
                'id': 'geocoding',
                'type': 'function',
                'function_call': {'name': 'geocoding', 'arguments': '{"address": "Vartak Nagar Thane"}'}
            }
        ]
    },
    'name': 'agent'
}

{
    'role': 'tool',
    'content': {
        'tool_response': {
            'id': 'geocoding',
            'name': 'geocoding',
            'content': '{"latitude":19.208258,"longitude":72.9610136}'
        }
    },
    'name': 'geocoding'
}

{
    'role': 'model',
    'content': {
        'tool_calls': [
            {
                'id': 'geodesic',
                'type': 'function',
                'function_call': {
                    'name': 'geodesic',
                    'arguments': '{"coordinate_1": {"latitude": 19.2083, "longitude": 72.961}, "coordinate_0": 
{"latitude": 20.2365, "longitude": 85.7557}}'
                }
            }
        ]
    },
    'name': 'agent'
}

{
    'role': 'tool',
    'content': {
        'tool_response': {
            'id': 'geodesic',
            'name': 'geodesic',
            'content': '{"geodesic_distance":1345.7381018159128,"unit":"km"}'
        }
    },
    'name': 'geodesic'
}

{
    'role': 'model',
    'content': {'text': 'The distance between Kalinga Vihar, Bubaneswar and Vartak Nagar Thane is 1345.74 km.'},
    'name': 'agent'
}

### Share and get paid for your tools
You can checkout the following examples to see how to create and share tools on XEntropy.
- [api](https://github.com/ckh112/xentropy/blob/main/docs/api-tool/notebook.ipynb)
- [agent](https://github.com/ckh112/xentropy/blob/main/docs/composite-agents-tool/notebook.ipynb)
- [rag](https://github.com/ckh112/xentropy/blob/main/docs/rag-tool/notebook.ipynb)

## Managing your account

The `Client` class has a few other utilities to access XEntropy platform.

In [7]:
from agentx.client import Client
client = Client(api_key=os.environ.get('XENTROPY_API_KEY'))

# See your account summary and usage data on each tool
client.summary()

{'id': 'jeremyk',
 'email': 'jeremy.kulcsar@diamondhill.io',
 'api_keys': {'default': 'cb7c7bd6b3104b719a6b8a8bf6433119'},
 'balance': 0.0,
 'payout': 0.0,
 'payment_link': 'https://buy.stripe.com/14kdR79oM1Ab2S4bIO',
 'ethereum_address': None}

In [8]:
# XEntropy credit can be used to pay for tool usage, if the tool is not free.
# You can top up your XEntropy credit by paying with credit card by navigating to the payment link.
print(client.summary().get('payment_link'))

https://buy.stripe.com/14kdR79oM1Ab2S4bIO


In [9]:
# Alternatively you can also top up your XEntropy credit by paying with cryptocurrency.
# We accept USDT, USDC, and DAI on the ethereum blockchain.
# First, register your address.
from eth_account import Account
account = Account.from_key('YOUR_PRIVATE_KEY')
# you can also use mnemonic phrase
# account = Account.from_mnemonic('YOUR_MNEMONIC_PHRASE')
response = client.register_ethereum_address(account=account)
print(response.json())

Error: Non-hexadecimal digit found

In [ ]:
# Sometimes you'd like to modify your published tool
# Example 1: Making the tool public to XEntropy platform user
client.modify_tool(
    tool="YOUR_TOOL_NAME",
    key="public",
    value=True,
)

In [ ]:
# Example 2: Tool description acts as a prompt to LLM, and hence it can be optimised through prompt engineering.
# You may want to modify it for improved performance.
# Check out https://github.com/microsoft/LMOps/tree/main/prompt_optimization for inspiration.
client.modify_tool(
    tool="YOUR_TOOL_NAME",
    key="description",
    value="YOU_NEW_DESCRIPTION",
)

In [ ]:
# You can take your tool away anytime.
client.delete_tool(
    tool="YOUR_TOOL_NAME"
)

XEntropy pays 80% of the tool-use revenue to tool developers's `payout` wallet. There is also a 5% bonus paid to the `balance` wallet to incentivize tool developers for consuming other tools on XEntropy.

At this moment we only support withdrawl on the ethereum network for selected stable coins. More withdrawal method is on the way.

In [ ]:
# To withdraw from your payout wallet
client.stable_coin_payout(
    amount=100000,  # amount to withdraw denominated in XEntropy Credit. i.e. 100000 XEntropy Credit = 1 USD
    address="YOUR_ETHEREUM_ADDRESS", # your ethereum network address
    stable_coin='USDT' # choose between ['USDT', 'USDC', 'DAI']
)